#### Fulfillment of Coursera projects

<h2> <u> Table of Contents </u> </h2>

##### [1. Question 1](#Question-1.)  

##### [2. Question 2](#Question-2)

#### Question 1.

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
html = requests.get(url) #getting html of the webpage

soup = BeautifulSoup(html.text) ## converting the html to soup element

In [5]:
table = soup.find_all('table', class_='wikitable sortable') ##finding table we need 

In [6]:
header = [x.text.replace('\n', '') for x in table[0].tr.find_all('th')]  # finding header
header

['Postal Code', 'Borough', 'Neighborhood']

In [7]:
#creating pandas data frame 
df = pd.DataFrame(columns=header)
data = {}
for tr in table[0].find_all('tr')[1:]:  #looping through 2 row to end
    td = [x.text.replace('\n', '') for x in tr.find_all('td')] #looping through all the table data in table row
    data['Postal Code'] = td[0]
    data['Borough'] = td[1]
    data['Neighborhood'] = td[2]
    df = df.append(data, ignore_index=True) #appending new row to existing data frame


In [8]:
df.columns = ['PostalCode', 'Borough', 'Neighborhood']  #renameing the columns

In [9]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [10]:
#removing columns with Borough value "Not assigned"
df_clean = df[df.Borough != 'Not assigned'] 

In [32]:
df_clean[list(df_clean.groupby(['PostalCode'])['Borough'].count() != 1)]  #no duplicate value of Postal Code

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [12]:
df_clean.reset_index(drop=True,inplace=True)

In [13]:
df_clean.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [14]:
print(f' There are {df_clean.shape[0]} rows in the data frame')

 There are 103 rows in the data frame


#### Question 2

In [15]:
import geocoder

In [16]:
# while(g.latlng is None):
#     g = geocoder.google('M5G, Toronto, Ontario')


###### Since geoCoder didn't work in my context.

In [17]:
latlong = pd.read_csv('Geospatial_Coordinates.csv')
latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
df_ = pd.merge(df_clean, latlong, how='inner', left_on='PostalCode', right_on='Postal Code', right_index=False)

In [19]:
df_clean = df_[['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]

In [20]:
df_clean.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


#### Question 3

In [42]:
#selecting Borough with Toronto
TorontoData = df_clean[df_clean.Borough.str.contains('Toronto')].reset_index(drop=True)

In [23]:
##credentials
client_id = 'IYCK1RQVEKTVPUPF5RWPJT1WUHUPAKIEBP0JBILTQORCE05V'
client_key='YRWSSCDZT10HFC0MXTOKAHCW0TL2WPKRKHQLKJ0UWBXJ2DSY'
version='20200718'

In [54]:
## finding venus around different postal code
lat, long = TorontoData.iloc[0, -2:].tolist()
radius = 500
limit = 100
url = f'https://api.foursquare.com/v2/venues/explore?&client_id={client_id}&client_secret={client_key}&v={version}&ll={lat},{long}&radius={radius}&limit={limit}'
url

'https://api.foursquare.com/v2/venues/explore?&client_id=IYCK1RQVEKTVPUPF5RWPJT1WUHUPAKIEBP0JBILTQORCE05V&client_secret=YRWSSCDZT10HFC0MXTOKAHCW0TL2WPKRKHQLKJ0UWBXJ2DSY&v=20200718&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [55]:
import requests

In [60]:
result = requests.get(url).json()

In [63]:
result['response'].keys()

dict_keys(['headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'totalResults', 'suggestedBounds', 'groups'])

In [83]:
result['response']['groups'][0]['items'][10]['venue']

{'id': '4c3e1eaa6faac9b66dc60d76',
 'name': 'Distillery Sunday Market',
 'location': {'address': '1 Trinity St',
  'lat': 43.650074989330655,
  'lng': -79.36183171318665,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.650074989330655,
    'lng': -79.36183171318665}],
  'distance': 475,
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['1 Trinity St', 'Toronto ON', 'Canada']},
 'categories': [{'id': '4bf58dd8d48988d1fa941735',
   'name': 'Farmers Market',
   'pluralName': 'Farmers Markets',
   'shortName': "Farmer's Market",
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_farmersmarket_',
    'suffix': '.png'},
   'primary': True}],
 'photos': {'count': 0, 'groups': []}}

In [79]:
nearvenus = pd.json_normalize(result['response']['groups'][0]['items'])

In [120]:
nearvenus.columns

Index(['referralId', 'reasons.count', 'reasons.items', 'venue.id',
       'venue.name', 'venue.location.address', 'venue.location.crossStreet',
       'venue.location.lat', 'venue.location.lng',
       'venue.location.labeledLatLngs', 'venue.location.distance',
       'venue.location.postalCode', 'venue.location.cc', 'venue.location.city',
       'venue.location.state', 'venue.location.country',
       'venue.location.formattedAddress', 'venue.categories',
       'venue.photos.count', 'venue.photos.groups', 'venue.venuePage.id',
       'venue.location.neighborhood'],
      dtype='object')

In [247]:
def extract_cat_name(row):
    try:
        cat_list = row['categories']
    except:
        cat_list = row['venue.categories']
    #print(cat_list)
    if len(cat_list) == 0:
        return None
    else:
        return cat_list[0]['name']
    

In [248]:

nearvenus.head(1).apply(extract_cat_name, axis=1)

0    Bakery
dtype: object

In [113]:
TorontoData.head(1)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636


In [262]:
def venueDataFrame(names, latitude, longitude, limit=100, radius=50):
    venue_df = pd.DataFrame(columns=['Name', 'Latitude', 'Longitude', 'Categories', 'PostalCode'])
    
    for name, latitude, longitude in zip(names, latitude, longitude):
        print(f'---{name}----')
        client_id = 'IYCK1RQVEKTVPUPF5RWPJT1WUHUPAKIEBP0JBILTQORCE05V'
        client_key='YRWSSCDZT10HFC0MXTOKAHCW0TL2WPKRKHQLKJ0UWBXJ2DSY'
        version='20200718'
    
        url = f'https://api.foursquare.com/v2/venues/explore?&client_id={client_id}&client_secret={client_key}&v={version}&ll={latitude},{longitude}&radius={radius}&limit={limit}'
    
        result = requests.get(url).json()['response']['groups'][0]['items']
       
        if len(result) != 0:
            ## convert into pandas data frame
            tempdf = pd.json_normalize(result)

            #print(tempdf.columns)

            tempdf['Categories'] = tempdf.apply(extract_cat_name, axis=1)
            #print(tempdf.columns.values)
            tempdf = tempdf[['venue.name', 'venue.location.lat', 'venue.location.lng', 'Categories']]

            tempdf['Postalcode'] = name

            tempdf.columns = venue_df.columns.values

            #print(tempdf.head())

            venue_df = venue_df.append(tempdf, ignore_index=True)

        
    return venue_df
    
    

In [263]:
TorontoData.head(1)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636


In [264]:
venus_dataframe = venueDataFrame(TorontoData['PostalCode'], TorontoData['Latitude'], TorontoData['Longitude'])

---M5A----
---M7A----
---M5B----
---M5C----
---M4E----


ConnectionError: HTTPSConnectionPool(host='api.foursquare.com', port=443): Max retries exceeded with url: /v2/venues/explore?&client_id=IYCK1RQVEKTVPUPF5RWPJT1WUHUPAKIEBP0JBILTQORCE05V&client_secret=YRWSSCDZT10HFC0MXTOKAHCW0TL2WPKRKHQLKJ0UWBXJ2DSY&v=20200718&ll=43.67635739999999,-79.2930312&radius=50&limit=100 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000000556072DCC8>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [271]:
CLIENT_ID = 'IYCK1RQVEKTVPUPF5RWPJT1WUHUPAKIEBP0JBILTQORCE05V'
CLIENT_SECRET='YRWSSCDZT10HFC0MXTOKAHCW0TL2WPKRKHQLKJ0UWBXJ2DSY'
VERSION='20200716'
LIMIT=100

In [272]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [273]:
manhattan_venues = getNearbyVenues(names=TorontoData['PostalCode'],
                                   latitudes=TorontoData['Latitude'],
                                   longitudes=TorontoData['Longitude']
                                  )

M5A


ConnectionError: HTTPSConnectionPool(host='api.foursquare.com', port=443): Max retries exceeded with url: /v2/venues/explore?&client_id=IYCK1RQVEKTVPUPF5RWPJT1WUHUPAKIEBP0JBILTQORCE05V&client_secret=YRWSSCDZT10HFC0MXTOKAHCW0TL2WPKRKHQLKJ0UWBXJ2DSY&v=20200716&ll=43.6542599,-79.3606359&radius=500&limit=100 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000005569715E08>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))